In [3]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import lightning as L
from trainer import LitModule, wandb_logger, profiler

c:\Users\Yinka\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\accelerator_connector.py:551: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))

    def forward(self, x):
        return self.l1(x)


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

    def forward(self, x):
        return self.l1(x)


class TESTAUTOENCODER(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x, y):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        return sample.view(-1), sample.view(-1)  # Reshape to match the input and output size

# Generate sample data
num_samples = 1000
input_dim = 28 * 28  # Adjusted to match the input size of the Encoder

# Create synthetic data (random numbers)
data = torch.randn(num_samples, input_dim)

# Create an instance of the CustomDataset
dataset = CustomDataset(data)

# Define batch size for DataLoader
batch_size = 64

# Create a DataLoader
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Test the DataLoader by iterating through a few batches
for batch_idx, (inputs, targets) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}:")
    print("Input shape:", inputs.shape)
    print("Target shape:", targets.shape)
    break  # Stop after the first batch for demonstration


Batch 1:
Input shape: torch.Size([64, 784])
Target shape: torch.Size([64, 784])


In [6]:
# model
test_model = TESTAUTOENCODER()
MODEL_ = LitModule(model=test_model, loss_criterion=F.mse_loss, metric=None)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rileydrizzy. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
c:\Users\Yinka\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [1]:
def model_w_func(model):
    # Access the model's state dictionary
    state_dict = model.state_dict()

    # Print the keys and shapes of the model's parameters
    for key, value in state_dict.items():
        print(f"Parameter: {key}, Size: {value.size()}")

    # Alternatively, you can directly print the state_dict
    print(state_dict)

In [ ]:
model_w_func(MODEL_)

In [8]:
#MODEL_.resume_training(from_wandb= True, MODEL_RUN_ID="model-mr8mxosy", VERSION="latest")

wandb:   1 of 1 files downloaded.  


In [5]:
# train model
trainer = L.Trainer(
    accelerator="auto",
    precision="16-mixed",
    logger=wandb_logger,
    profiler=profiler,
    strategy="auto",
    check_val_every_n_epoch=5,
    max_epochs=10,
)

trainer.fit(
    model=MODEL_,
    train_dataloaders=train_loader,
    val_dataloaders=train_loader, ckpt_path = 
)

c:\Users\Yinka\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\accelerator_connector.py:551: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Yinka\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\utilities\parsing.py:43: attribute 'model' removed from hparams because it cannot be pickled
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
c:\Users\Yinka\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:639: Checkpoint directory C:\Main Workspace\Cohort8-Ransom-Kuti-Ladipo\sig

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Yinka\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
c:\Users\Yinka\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
c:\Users\Yinka\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
c:\Users\Yinka\AppData\Local\Programs\Python\Python311\Lib\site-p

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


# BREAK